# Flight Delay Prediction

## Flight Data Pre-processing

- Exploring <strong>2016_1.csv</strong> flight Data to generalize
- Loading only the required columns
- Visualizing the missing data
- Removing the rows that have missing data in their columns
- Combining all the flight data after data handling into a df
- Storing the preprocessed data as a dataframe


In [70]:
#importing required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as px
import missingno as msno
%matplotlib inline


In [71]:
#importing the data
airport_data = pd.read_csv(r'D:\git_project\Flight-Delay-Prediction\Data\Flight\airports.csv')
flight_2016 = pd.read_csv(r'D:\git_project\Flight-Delay-Prediction\Data\Flight\2016.csv')

In [73]:
#to see all the columns of data frame
pd.set_option("display.max_columns", None)

In [75]:
#data overview
flight_2016.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2016-01-01,DL,1248,DTW,LAX,1935,1935.0,0.0,23.0,1958.0,2107.0,13.0,2144,2120.0,-24.0,0.0,NaN,0.0,309.0,285.0,249.0,1979.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-01,DL,1251,ATL,GRR,2125,2130.0,5.0,13.0,2143.0,2315.0,4.0,2321,2319.0,-2.0,0.0,NaN,0.0,116.0,109.0,92.0,640.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-01,DL,1254,LAX,ATL,2255,2256.0,1.0,19.0,2315.0,542.0,5.0,600,547.0,-13.0,0.0,NaN,0.0,245.0,231.0,207.0,1947.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-01,DL,1255,SLC,ATL,1656,1700.0,4.0,12.0,1712.0,2205.0,8.0,2229,2213.0,-16.0,0.0,NaN,0.0,213.0,193.0,173.0,1590.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-01-01,DL,1256,BZN,MSP,900,1012.0,72.0,63.0,1115.0,1416.0,4.0,1216,1420.0,124.0,0.0,NaN,0.0,136.0,188.0,121.0,874.0,72.0,0.0,52.0,0.0,0.0,NaN


### Visualizing Missing Data
- White lines indicate missing values
- There is an <strong>interesting trend</strong> in the columns that have missing values
- They mostly have missing values in common rows namely: 
    - <strong>DepTime, DepDelayMinutes and, DepDel15</strong>
    - <strong>ArrTime, ArrDelayMinutes and, ArrDel15</strong>

In [77]:
print(msno.matrix(flight_2016))

Axes(0.125,0.11;0.698618x0.77)


In [76]:
#Finding the null values present in different columns
flight_2016.isnull().sum()

FL_DATE                      0
OP_CARRIER                   0
OP_CARRIER_FL_NUM            0
ORIGIN                       0
DEST                         0
CRS_DEP_TIME                 0
DEP_TIME                 63456
DEP_DELAY                63456
TAXI_OUT                 65418
WHEELS_OFF               65418
WHEELS_ON                67844
TAXI_IN                  67844
CRS_ARR_TIME                 0
ARR_TIME                 67844
ARR_DELAY                79513
CANCELLED                    0
CANCELLATION_CODE      5551797
DIVERTED                     0
CRS_ELAPSED_TIME             6
ACTUAL_ELAPSED_TIME      79513
AIR_TIME                 79513
DISTANCE                     0
CARRIER_DELAY          4653419
WEATHER_DELAY          4653419
NAS_DELAY              4653419
SECURITY_DELAY         4653419
LATE_AIRCRAFT_DELAY    4653419
Unnamed: 27            5617658
dtype: int64

In [8]:
columns_to_drop_flight_data = [
    'CANCELLATION_CODE','CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 
    'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'Unnamed: 27'
]
flight_2016= flight_2016.drop(columns=columns_to_drop_flight_data, errors='ignore')

In [9]:
flight_2016.shape

(5617658, 21)

In [10]:
flight_2016 = flight_2016.dropna( axis=0)

In [9]:
flight_2016.shape

(5538145, 21)

In [11]:
flight_2016.isnull().sum()

FL_DATE                0
OP_CARRIER             0
OP_CARRIER_FL_NUM      0
ORIGIN                 0
DEST                   0
CRS_DEP_TIME           0
DEP_TIME               0
DEP_DELAY              0
TAXI_OUT               0
WHEELS_OFF             0
WHEELS_ON              0
TAXI_IN                0
CRS_ARR_TIME           0
ARR_TIME               0
ARR_DELAY              0
CANCELLED              0
DIVERTED               0
CRS_ELAPSED_TIME       0
ACTUAL_ELAPSED_TIME    0
AIR_TIME               0
DISTANCE               0
dtype: int64

In [12]:
airport_data.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [13]:
airport_data.isnull().sum()

IATA_CODE    0
AIRPORT      0
CITY         0
STATE        0
COUNTRY      0
LATITUDE     3
LONGITUDE    3
dtype: int64

In [14]:
airport_data = airport_data.dropna()

In [62]:
flight_2016.shape

(5538145, 21)

In [63]:
flight_df.shape

(5527939, 25)

In [17]:
weather_data['StartTime(UTC)'] = pd.to_datetime(weather_data['StartTime(UTC)'])
weather_data['EndTime(UTC)'] = pd.to_datetime(weather_data['EndTime(UTC)'])

In [18]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8627181 entries, 0 to 8627180
Data columns (total 14 columns):
 #   Column             Dtype         
---  ------             -----         
 0   EventId            object        
 1   Type               object        
 2   Severity           object        
 3   StartTime(UTC)     datetime64[ns]
 4   EndTime(UTC)       datetime64[ns]
 5   Precipitation(in)  float64       
 6   TimeZone           object        
 7   AirportCode        object        
 8   LocationLat        float64       
 9   LocationLng        float64       
 10  City               object        
 11  County             object        
 12  State              object        
 13  ZipCode            float64       
dtypes: datetime64[ns](2), float64(4), object(8)
memory usage: 921.5+ MB


In [16]:
airport_df = airport_data.copy()
flight_df = flight_2016.copy()

In [17]:
airport_df.columns = ['ORIGIN', 'ORIGIN_AIRPORT', 'ORIGIN_CITY', 'ORIGIN_STATE', 'ORIGIN_COUNTRY', 'ORIGIN_LATITUDE', 'ORIGIN_LONGITUDE']
flight_df = flight_df.merge(airport_df, on='ORIGIN', how='left')

In [18]:
flight_df.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,ORIGIN_AIRPORT,ORIGIN_CITY,ORIGIN_STATE,ORIGIN_COUNTRY,ORIGIN_LATITUDE,ORIGIN_LONGITUDE
0,2016-01-01,DL,1248,DTW,LAX,1935,1935.0,0.0,23.0,1958.0,2107.0,13.0,2144,2120.0,-24.0,0.0,0.0,309.0,285.0,249.0,1979.0,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884
1,2016-01-01,DL,1251,ATL,GRR,2125,2130.0,5.0,13.0,2143.0,2315.0,4.0,2321,2319.0,-2.0,0.0,0.0,116.0,109.0,92.0,640.0,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694
2,2016-01-01,DL,1254,LAX,ATL,2255,2256.0,1.0,19.0,2315.0,542.0,5.0,600,547.0,-13.0,0.0,0.0,245.0,231.0,207.0,1947.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807
3,2016-01-01,DL,1255,SLC,ATL,1656,1700.0,4.0,12.0,1712.0,2205.0,8.0,2229,2213.0,-16.0,0.0,0.0,213.0,193.0,173.0,1590.0,Salt Lake City International Airport,Salt Lake City,UT,USA,40.78839,-111.97777
4,2016-01-01,DL,1256,BZN,MSP,900,1012.0,72.0,63.0,1115.0,1416.0,4.0,1216,1420.0,124.0,0.0,0.0,136.0,188.0,121.0,874.0,Bozeman Yellowstone International Airport (Gal...,Bozeman,MT,USA,45.77690,-111.15301


In [19]:
airport_df.head()

,ORIGIN,ORIGIN_AIRPORT,ORIGIN_CITY,ORIGIN_STATE,ORIGIN_COUNTRY,ORIGIN_LATITUDE,ORIGIN_LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [20]:
airport_df.columns = ['DEST', 'DEST_AIRPORT', 'DEST_CITY', 'DEST_STATE', 'DEST_COUNTRY', 'DEST_LATITUDE', 'DEST_LONGITUDE']
flight_df = flight_df.merge(airport_df, on='DEST', how='left')

In [21]:
flight_df.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,ORIGIN_AIRPORT,ORIGIN_CITY,ORIGIN_STATE,ORIGIN_COUNTRY,ORIGIN_LATITUDE,ORIGIN_LONGITUDE,DEST_AIRPORT,DEST_CITY,DEST_STATE,DEST_COUNTRY,DEST_LATITUDE,DEST_LONGITUDE
0,2016-01-01,DL,1248,DTW,LAX,1935,1935.0,0.0,23.0,1958.0,2107.0,13.0,2144,2120.0,-24.0,0.0,0.0,309.0,285.0,249.0,1979.0,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807
1,2016-01-01,DL,1251,ATL,GRR,2125,2130.0,5.0,13.0,2143.0,2315.0,4.0,2321,2319.0,-2.0,0.0,0.0,116.0,109.0,92.0,640.0,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694,Gerald R. Ford International Airport,Grand Rapids,MI,USA,42.88082,-85.52277
2,2016-01-01,DL,1254,LAX,ATL,2255,2256.0,1.0,19.0,2315.0,542.0,5.0,600,547.0,-13.0,0.0,0.0,245.0,231.0,207.0,1947.0,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694
3,2016-01-01,DL,1255,SLC,ATL,1656,1700.0,4.0,12.0,1712.0,2205.0,8.0,2229,2213.0,-16.0,0.0,0.0,213.0,193.0,173.0,1590.0,Salt Lake City International Airport,Salt Lake City,UT,USA,40.78839,-111.97777,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694
4,2016-01-01,DL,1256,BZN,MSP,900,1012.0,72.0,63.0,1115.0,1416.0,4.0,1216,1420.0,124.0,0.0,0.0,136.0,188.0,121.0,874.0,Bozeman Yellowstone International Airport (Gal...,Bozeman,MT,USA,45.77690,-111.15301,Minneapolis-Saint Paul International Airport,Minneapolis,MN,USA,44.88055,-93.21692


In [22]:
flight_df.isnull().sum()

FL_DATE                   0
OP_CARRIER                0
OP_CARRIER_FL_NUM         0
ORIGIN                    0
DEST                      0
CRS_DEP_TIME              0
DEP_TIME                  0
DEP_DELAY                 0
TAXI_OUT                  0
WHEELS_OFF                0
WHEELS_ON                 0
TAXI_IN                   0
CRS_ARR_TIME              0
ARR_TIME                  0
ARR_DELAY                 0
CANCELLED                 0
DIVERTED                  0
CRS_ELAPSED_TIME          0
ACTUAL_ELAPSED_TIME       0
AIR_TIME                  0
DISTANCE                  0
ORIGIN_AIRPORT         5100
ORIGIN_CITY            5100
ORIGIN_STATE           5100
ORIGIN_COUNTRY         5100
ORIGIN_LATITUDE        5100
ORIGIN_LONGITUDE       5100
DEST_AIRPORT           5106
DEST_CITY              5106
DEST_STATE             5106
DEST_COUNTRY           5106
DEST_LATITUDE          5106
DEST_LONGITUDE         5106
dtype: int64

In [23]:
#removing the rows with no origin or dest airport code
flight_df = flight_df.dropna()

In [24]:
flight_df.shape

(5527939, 33)

In [30]:
irrelavant_cols = ['ORIGIN_AIRPORT', 'ORIGIN_CITY', 'ORIGIN_STATE','ORIGIN_COUNTRY',
                   'DEST_AIRPORT','DEST_CITY', 'DEST_STATE', 'DEST_COUNTRY']

flight_df = flight_df.drop(columns = irrelavant_cols,axis =0)

* Data Dictionary

| Column Name | Description |
|---|---|
| FL_DATE | Date of the flight (YYYY-MM-DD) |
| OP_CARRIER | Operating Carrier |
| CARRIER | Marketing Carrier |
| FL_NUM | Flight Number |
| ORIGIN | Origin Airport Code |
| DEST | Destination Airport Code |
| CRS_DEP_TIME | Scheduled Departure Time (24-hour format) |
| DEP_TIME | Actual Departure Time (24-hour format) |
| DEP_DELAY | Departure Delay (minutes) |
| TAXI_OUT | Taxi Out Time (minutes) |
| WHEELS_OFF | Wheels Off Time (24-hour format) |
| WHEELS_ON | Wheels On Time (24-hour format) |
| TAXI_IN | Taxi In Time (minutes) |
| CRS_ARR_TIME | Scheduled Arrival Time (24-hour format) |
| ARR_TIME | Actual Arrival Time (24-hour format) |
| ARR_DELAY | Arrival Delay (minutes) |
| CANCELLED | Cancelled (0 - Not Cancelled, 1 - Cancelled) |
| DIVERTED | Diverted (0 - Not Diverted, 1 - Diverted) |
| CRS_ELAPSED_TIME | Scheduled Elapsed Time (minutes) |
| ACTUAL_ELAPSED_TIME | Actual Elapsed Time (minutes) |
| AIR_TIME | Air Time (minutes) |
| DISTANCE | Distance (miles) |
| ORIGIN_LATITUDE | Origin Airport Latitude |
| ORIGIN_LONGITUDE | Origin Airport Longitude |
| DEST_LATITUDE | Destination Airport Latitude |
| DEST_LONGITUDE | Destination Airport Longitude |


In [68]:
flight_df.head(10)

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,ORIGIN_LATITUDE,ORIGIN_LONGITUDE,DEST_LATITUDE,DEST_LONGITUDE
0,2016-01-01,DL,1248,DTW,LAX,1935,1935.0,0.0,23.0,1958.0,2107.0,13.0,2144,2120.0,-24.0,0.0,0.0,309.0,285.0,249.0,1979.0,42.21206,-83.34884,33.94254,-118.40807
1,2016-01-01,DL,1251,ATL,GRR,2125,2130.0,5.0,13.0,2143.0,2315.0,4.0,2321,2319.0,-2.0,0.0,0.0,116.0,109.0,92.0,640.0,33.64044,-84.42694,42.88082,-85.52277
2,2016-01-01,DL,1254,LAX,ATL,2255,2256.0,1.0,19.0,2315.0,542.0,5.0,600,547.0,-13.0,0.0,0.0,245.0,231.0,207.0,1947.0,33.94254,-118.40807,33.64044,-84.42694
3,2016-01-01,DL,1255,SLC,ATL,1656,1700.0,4.0,12.0,1712.0,2205.0,8.0,2229,2213.0,-16.0,0.0,0.0,213.0,193.0,173.0,1590.0,40.78839,-111.97777,33.64044,-84.42694
4,2016-01-01,DL,1256,BZN,MSP,900,1012.0,72.0,63.0,1115.0,1416.0,4.0,1216,1420.0,124.0,0.0,0.0,136.0,188.0,121.0,874.0,45.77690,-111.15301,44.88055,-93.21692
5,2016-01-01,DL,1257,ATL,BNA,1233,1356.0,83.0,22.0,1418.0,1356.0,6.0,1239,1402.0,83.0,0.0,0.0,66.0,66.0,38.0,214.0,33.64044,-84.42694,36.12448,-86.67818
6,2016-01-01,DL,1257,BNA,ATL,1320,1446.0,86.0,15.0,1501.0,1638.0,6.0,1530,1644.0,74.0,0.0,0.0,70.0,58.0,37.0,214.0,36.12448,-86.67818,33.64044,-84.42694
7,2016-01-01,DL,1258,ATL,JAX,945,946.0,1.0,19.0,1005.0,1050.0,3.0,1050,1053.0,3.0,0.0,0.0,65.0,67.0,45.0,270.0,33.64044,-84.42694,30.49406,-81.68786
8,2016-01-01,DL,1258,JAX,ATL,1145,1144.0,-1.0,12.0,1156.0,1239.0,8.0,1302,1247.0,-15.0,0.0,0.0,77.0,63.0,43.0,270.0,30.49406,-81.68786,33.64044,-84.42694
9,2016-01-01,DL,1259,ATL,OKC,2110,2107.0,-3.0,16.0,2123.0,2219.0,5.0,2236,2224.0,-12.0,0.0,0.0,146.0,137.0,116.0,761.0,33.64044,-84.42694,35.39309,-97.60073


In [35]:
flight_df['FL_DATE'] = pd.to_datetime(flight_df['FL_DATE'])

In [67]:
flight_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5527939 entries, 0 to 5538144
Data columns (total 25 columns):
 #   Column               Dtype         
---  ------               -----         
 0   FL_DATE              datetime64[ns]
 1   OP_CARRIER           object        
 2   OP_CARRIER_FL_NUM    int64         
 3   ORIGIN               object        
 4   DEST                 object        
 5   CRS_DEP_TIME         object        
 6   DEP_TIME             float64       
 7   DEP_DELAY            float64       
 8   TAXI_OUT             float64       
 9   WHEELS_OFF           float64       
 10  WHEELS_ON            float64       
 11  TAXI_IN              float64       
 12  CRS_ARR_TIME         int64         
 13  ARR_TIME             float64       
 14  ARR_DELAY            float64       
 15  CANCELLED            float64       
 16  DIVERTED             float64       
 17  CRS_ELAPSED_TIME     float64       
 18  ACTUAL_ELAPSED_TIME  float64       
 19  AIR_TIME             